# 04 - Deep MLP
Train the Deep MLP and visualize results.


In [1]:
from pathlib import Path
import sys

ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd


In [2]:
import importlib
from pathlib import Path

import src.models as models
importlib.reload(models)
make_mlp_model = models.make_mlp_model
make_pipeline = models.make_pipeline
build_search = models.build_search
from src.eval import evaluate_models, compute_full_metrics
from src.plots import plot_actual_vs_pred, plot_error_distribution
from _common import load_dataset, prepare_features, ROOT
from src.split import SplitConfig
from contextlib import contextmanager

SEED = 42

SMALL_MODE = False  # toggle for quick iteration
TUNE_MODE = "fast"  # off | fast | full
SEARCH_VERBOSE = 2  # sklearn CV logging
SEARCH_N_ITER = None  # only used for randomized search
SHOW_CV_TQDM = True  # tqdm progress for CV fits

# MLflow
MLFLOW_ENABLED = True
MLFLOW_EXPERIMENT = "f1-laptime"
MLFLOW_TRACKING_URI = (ROOT / "mlruns").as_uri()
MLFLOW_RUN_NAME = "mlp_notebook"

# Verify MLflow availability
if MLFLOW_ENABLED:
    try:
        import mlflow  # noqa: F401
        print(f"MLflow available: {mlflow.__version__}")
    except Exception:
        print("MLflow not installed; set MLFLOW_ENABLED=False or install mlflow.")
        MLFLOW_ENABLED = False

# Model saving
SAVE_MODEL = True
MODEL_PATH = ROOT / "reports" / "models" / "deep_mlp.joblib"

MLP_VERBOSE = 2  # 0/False, 1=tqdm, 2=tqdm + per-epoch log
MLP_LOG_EVERY = 1
MLP_BATCH_LOG_EVERY = 20  # set 0 to disable
MLP_LIVE_PLOT_EVERY = 5   # epochs; set 0 to disable

# Small-mode overrides / base hyperparameters
MLP_HIDDEN_LAYERS = None
MLP_EPOCHS = None
MLP_BATCH_SIZE = None
MLP_DROPOUT = None
MLP_LR = None
MLP_WEIGHT_DECAY = None
MLP_PARAM_GRID = None
if SMALL_MODE:
    MLP_HIDDEN_LAYERS = (128, 64)
    MLP_EPOCHS = 40
    MLP_BATCH_SIZE = 64
    SEARCH_N_ITER = 6
    MLP_PARAM_GRID = {
        "model__hidden_layers": [(128, 64), (128, 64, 32)],
        "model__dropout": [0.2, 0.3],
        "model__lr": [1e-3],
        "model__batch_size": [64, 128],
        "model__epochs": [30, 60],
        "model__weight_decay": [0.0, 1e-4],
    }

# Optional GPU info + explicit device
# DEVICE can be: "auto", "cpu", "cuda"
DEVICE = "auto"

device_resolved = None
if DEVICE in ("cpu", "cuda"):
    device_resolved = DEVICE
    print(f"Device forced by user: {device_resolved}")
elif DEVICE != "auto":
    print(f"Unknown DEVICE value '{DEVICE}', falling back to auto.")

if device_resolved is None:
    try:
        import torch
        if torch.cuda.is_available():
            device_resolved = "cuda"
            print(f"GPU available: {torch.cuda.get_device_name(0)}")
        else:
            device_resolved = "cpu"
            print("GPU not available; using CPU.")
    except Exception as exc:
        print(f"Torch not available for GPU check ({exc}).")

DEVICE = device_resolved
@contextmanager
def tqdm_joblib(total, desc="CV fits"):
    try:
        import joblib
        from tqdm.auto import tqdm
    except Exception:  # noqa: BLE001
        yield None
        return

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            try:
                tqdm_bar.update(n=self.batch_size)
            except Exception:
                pass
            return super().__call__(*args, **kwargs)

    tqdm_bar = tqdm(total=total, desc=desc)
    old_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_bar
    finally:
        joblib.parallel.BatchCompletionCallBack = old_callback
        tqdm_bar.close()


def estimate_total_fits(search):
    try:
        n_splits = search.cv.get_n_splits()
    except Exception:
        n_splits = getattr(search.cv, "n_splits", 1)
    if hasattr(search, "n_iter"):
        n_candidates = search.n_iter
    else:
        try:
            from sklearn.model_selection import ParameterGrid
            n_candidates = len(list(ParameterGrid(search.param_grid)))
        except Exception:
            n_candidates = 1
    return n_splits * n_candidates


split_config = SplitConfig(test_rounds=None)
df, metadata = load_dataset()
train_df, val_df, trainval_df, test_df, features = prepare_features(df, metadata, split_config=split_config)

X_train = train_df[features]
y_train = train_df["LapTimeSeconds"].to_numpy()
X_val = val_df[features]
y_val = val_df["LapTimeSeconds"].to_numpy()

base = make_pipeline(
    make_mlp_model(
        SEED,
        verbose=MLP_VERBOSE,
        log_every=MLP_LOG_EVERY,
        log_batch_every=MLP_BATCH_LOG_EVERY,
        live_plot_every=MLP_LIVE_PLOT_EVERY,
        hidden_layers=MLP_HIDDEN_LAYERS,
        epochs=MLP_EPOCHS,
        batch_size=MLP_BATCH_SIZE,
        dropout=MLP_DROPOUT,
        lr=MLP_LR,
        weight_decay=MLP_WEIGHT_DECAY,
        device=DEVICE,
    ),
    features,
)
model = build_search(
    "Deep MLP",
    base,
    random_state=SEED,
    mode=TUNE_MODE,
    param_grid=MLP_PARAM_GRID,
    n_iter=SEARCH_N_ITER,
    search_verbose=SEARCH_VERBOSE,
)

if SHOW_CV_TQDM and hasattr(model, "cv"):
    total_fits = estimate_total_fits(model)
    with tqdm_joblib(total_fits, desc="CV fits"):
        metrics, preds, fitted = evaluate_models({"Deep MLP": model}, X_train, y_train, X_val, y_val)
else:
    metrics, preds, fitted = evaluate_models({"Deep MLP": model}, X_train, y_train, X_val, y_val)

metrics



                                          
Models: 100%|██████████| 1/1 [13:37<00:00, 817.09s/it]

Deep MLP -> MAE: 0.9548, R2: 0.9781 (817.1s)
Deep MLP best_params: {'model__weight_decay': 0.0, 'model__lr': 0.0005, 'model__hidden_layers': (256, 128, 64), 'model__epochs': 100, 'model__dropout': 0.2, 'model__batch_size': 128}


,mae,rmse,r2,model
0,0.954834,1.494937,0.978113,Deep MLP


In [3]:
best = fitted["Deep MLP"].best_estimator_ if hasattr(fitted["Deep MLP"], "best_estimator_") else fitted["Deep MLP"]
X_trainval = trainval_df[features]
y_trainval = trainval_df["LapTimeSeconds"].to_numpy()
X_test = test_df[features]
y_test = test_df["LapTimeSeconds"].to_numpy()
best.fit(X_trainval, y_trainval)
test_pred = best.predict(X_test)

plot_actual_vs_pred(y_test, test_pred, title="Deep MLP: Predicted vs Actual")


  batch 0020 | loss=3.6154
  batch 0040 | loss=3.6403
  batch 0060 | loss=3.8297
  batch 0080 | loss=3.5425
  batch 0100 | loss=3.6010
  batch 0120 | loss=3.5534
  batch 0140 | loss=3.0016
  batch 0160 | loss=3.5647
  batch 0180 | loss=3.5475
  batch 0200 | loss=3.7717
  batch 0220 | loss=3.5564
  batch 0240 | loss=3.3604
  batch 0260 | loss=4.0847
  batch 0280 | loss=3.1306
  batch 0300 | loss=3.7180
  batch 0320 | loss=3.6716
  batch 0340 | loss=4.1384
  batch 0360 | loss=3.3129
  batch 0380 | loss=3.6720
  batch 0400 | loss=3.7176
  batch 0420 | loss=3.6621
Epoch 66/100 - Train: 3.6370 | Val: 0.3233 | LR: 3.1e-05
  batch 0020 | loss=3.6085
  batch 0040 | loss=4.0476
  batch 0060 | loss=3.2688
  batch 0080 | loss=4.3878
  batch 0100 | loss=3.5074
  batch 0120 | loss=3.6089
  batch 0140 | loss=3.6839
  batch 0160 | loss=3.4915
  batch 0180 | loss=3.3964
  batch 0200 | loss=3.8089
  batch 0220 | loss=3.5514
  batch 0240 | loss=3.2354
  batch 0260 | loss=3.8274
  batch 0280 | loss=3.891

In [4]:
plot_error_distribution(y_test, test_pred, title="Deep MLP: Residuals")


In [5]:
# Training history (post-hoc)
import plotly.express as px

est = fitted["Deep MLP"]
if hasattr(est, "best_estimator_"):
    est = est.best_estimator_
model = est.named_steps["model"]
hist = getattr(model, "training_history_", None)
if hist:
    df_hist = pd.DataFrame(hist)
    fig = px.line(df_hist, y=["train_loss", "val_loss"], title="Deep MLP Training Curves")
    fig
else:
    print("No training history found.")


In [6]:
import joblib

# Test metrics (full 2025 season)
mlp_test_metrics = compute_full_metrics(y_test, test_pred, n_features=len(features))
mlp_test_metrics

# Save model for inference
if SAVE_MODEL:
    MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(best, MODEL_PATH)
    print("Saved model to {}".format(MODEL_PATH))

# MLflow logging
if MLFLOW_ENABLED:
    try:
        import mlflow
    except ImportError:
        print("MLflow not installed; skipping MLflow logging.")
    else:
        def _coerce_params(params):
            return {k: str(v) for k, v in params.items()}

        if MLFLOW_TRACKING_URI:
            mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
        mlflow.set_experiment(MLFLOW_EXPERIMENT)
        with mlflow.start_run(run_name=MLFLOW_RUN_NAME or "mlp_notebook"):
            mlflow.log_param("tune_mode", TUNE_MODE)
            mlflow.log_param("small_mode", SMALL_MODE)
            mlflow.log_param("test_rounds", split_config.test_rounds)

            if hasattr(fitted["Deep MLP"], "best_params_"):
                mlflow.log_params(_coerce_params(fitted["Deep MLP"].best_params_))
            else:
                base_params = {
                    "hidden_layers": MLP_HIDDEN_LAYERS,
                    "dropout": MLP_DROPOUT,
                    "lr": MLP_LR,
                    "epochs": MLP_EPOCHS,
                    "batch_size": MLP_BATCH_SIZE,
                    "weight_decay": MLP_WEIGHT_DECAY,
                }
                mlflow.log_params(_coerce_params({k: v for k, v in base_params.items() if v is not None}))

            row = metrics[metrics["model"] == "Deep MLP"].iloc[0]
            for metric in ("mae", "rmse", "r2"):
                mlflow.log_metric("val_" + metric, float(row[metric]))
            for metric in ("mae", "rmse", "r2", "mape_pct", "smape_pct"):
                if metric in mlp_test_metrics:
                    mlflow.log_metric("test_" + metric, float(mlp_test_metrics[metric]))

            if SAVE_MODEL and MODEL_PATH.exists():
                mlflow.log_artifact(str(MODEL_PATH), artifact_path="models")


Saved model to C:\Users\tvcar\Desktop\FOM\2. Semester\Maschinelles Lernen\ml_f1\reports\models\deep_mlp.joblib


c:\Users\tvcar\Desktop\FOM\2. Semester\Maschinelles Lernen\ml_f1\.venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning:

The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.

